In [1]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from langchain.retrievers import BM25Retriever, Ense`m`bleRetriever


In [2]:
load_dotenv()

True

In [3]:
# loaders = [
#     PDFPlumberLoader("data/RAG/Adaptive_RAG.pdf"),
#     PDFPlumberLoader("data/RAG/Naive_RAG.pdf"),
#     PDFPlumberLoader("data/RAG/RAPTOR_RAG.pdf"),
#     PDFPlumberLoader("data/RAG/Self_RAG.pdf"),
# ]

In [4]:
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

# len(docs)

In [5]:
loader = PDFPlumberLoader("data/RAG/Self_RAG.pdf")
docs = loader.load()

In [6]:
text_splitters = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitters.split_documents(docs)

In [7]:
len(documents)

110

In [8]:
embeddings = HuggingFaceEndpointEmbeddings(
    model="BAAI/bge-m3",
    task="feature-extraction"
)


In [9]:
embeddings.embed_query("안녕")

[-0.015962200239300728,
 0.03353027626872063,
 -0.02759610116481781,
 -0.02196122705936432,
 -0.04239477217197418,
 -0.023451130837202072,
 -0.02042667753994465,
 -0.03987332060933113,
 0.01880594529211521,
 0.01210780069231987,
 0.03275774419307709,
 0.012041877955198288,
 0.019448615610599518,
 0.004909968003630638,
 0.04015949368476868,
 -0.043033417314291,
 0.031329624354839325,
 -0.023474346846342087,
 -0.018658410757780075,
 -0.033938225358724594,
 -0.0285493154078722,
 -0.010305242612957954,
 0.04746466875076294,
 0.000241091416683048,
 0.006823814939707518,
 0.02875480428338051,
 -0.032485101372003555,
 0.00893373042345047,
 -0.004612055141478777,
 -0.020030664280056953,
 0.035180751234292984,
 0.03845236822962761,
 -0.016920989379286766,
 -0.0347713828086853,
 -0.03291317820549011,
 -0.04001068323850632,
 0.0009524685447104275,
 -0.008383507840335369,
 -0.03556610271334648,
 0.04304749146103859,
 0.039022184908390045,
 -0.006132516544312239,
 0.006225868593901396,
 -0.04443933

In [10]:
vector_store = FAISS.from_documents(documents, embeddings)

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/pipeline/feature-extraction/BAAI/bge-m3 (Request ID: KJuv2dKD-aj6HZLPkeepw)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
llm = ChatOllama(model = "llama3.1")

In [ ]:
template = """
당신은 QA답변 AI 어시스턴스입니다. 주어진 내용을 참고해 답변에 답변해 주세요.
답변을 모르는 경우에는, 모른다고 해주세요.
답변은 한국어로 해주세요.
답변에는 출처를 적어주세요.

#Question:
{question}
#Context:
{context}

#Answer:
"""
prompt = PromptTemplate.from_template(template=template)

In [ ]:
chain = ({"context":retriever, "question":RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [ ]:
chain.invoke("self-rag에 대해서 설명해 주세요")